<a href="https://colab.research.google.com/github/orodriguezq/orodriguezq-Senales_Y_Sistemas/blob/main/Talleres/AM_Modulaci%C3%B3n_y_Demodulaci%C3%B3n_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar dependencias

In [37]:
!pip install -q streamlit yt-dlp pydub joblib scikit-learn matplotlib requests librosa
!apt update && apt install -y ffmpeg
!ffmpeg -version


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
35 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

Eliminar scripts viejos

In [38]:
!rm -f app.py  # o el nombre de tu script anterior
!rm -f logs.txt cloudflared.log


Crear el script app.py

In [45]:
%%bash
cat > app.py << 'EOF'
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import io
import os
from scipy.io import wavfile
from yt_dlp import YoutubeDL
from pydub import AudioSegment
import traceback

# --- Configuración de la app ---
st.set_page_config(page_title="AM Modulación/Demodulación", page_icon="📻", layout="wide")
st.title("AM Modulación y Demodulación Coherente")

# --- Descargar audio de YouTube ---
@st.cache_data
def download_youtube_audio(url: str, out_fname: str = 'audio.wav'):
    for f in os.listdir():
        if f.startswith('audio.') or f == out_fname:
            try: os.remove(f)
            except: pass
    opts = {'format':'bestaudio/best','outtmpl':'audio.%(ext)s','quiet':True}
    with YoutubeDL(opts) as ydl:
        info = ydl.extract_info(url, download=True)
        ext  = info.get('ext','m4a')
        fname= f"audio.{ext}"
    audio = AudioSegment.from_file(fname).set_channels(1)
    audio.export(out_fname, format='wav')
    os.remove(fname)
    return out_fname

# --- Dominio tiempo & frecuencia ---
def plot_time_freq(x, fs, title, t_lim=None, f_lim=None):
    N = len(x); t = np.arange(N)/fs
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,3))
    ax1.plot(t,x); ax1.set(xlabel='t [s]',ylabel='x(t)',title=f'{title} – Tiempo')
    if t_lim: ax1.set_xlim(t_lim)
    Xf = np.fft.fft(x); freqs = np.fft.fftfreq(N,1/fs)
    ax2.plot(freqs,np.abs(Xf)/N); ax2.set(xlabel='f [Hz]',ylabel='|X(f)|',title=f'{title} – Frecuencia')
    if f_lim: ax2.set_xlim(f_lim)
    fig.tight_layout(); st.pyplot(fig)

# --- Parámetros ---
st.sidebar.header("Parámetros")
url = st.sidebar.text_input("URL de YouTube","https://www.youtube.com/watch?v=dQw4w9WgXcQ")
t0  = st.sidebar.number_input("Tiempo inicio (s)",min_value=0.0,value=0.0,step=0.1)
t1  = st.sidebar.number_input("Tiempo fin    (s)",min_value=0.1,value=1.0,step=0.1)
f0  = st.sidebar.number_input("Portadora f0 (Hz)",min_value=100.0,value=10000.0,step=100.0)
B   = st.sidebar.number_input("LPF ancho B (Hz)",min_value=100.0,value=5000.0,step=100.0)

if url:
    try:
        # descarga y WAV mono
        wav_file = download_youtube_audio(url)
        fs,data = wavfile.read(wav_file)
        if data.ndim>1: data=data[:,0]
        audio_seg = AudioSegment.from_wav(wav_file).set_channels(1)

        # 1) mensaje
        ms0,ms1 = int(t0*1000),int(t1*1000)
        msg = audio_seg[ms0:ms1]
        m   = np.array(msg.get_array_of_samples()); Ac=np.max(np.abs(m))
        st.subheader("1) Mensaje m(t)")
        plot_time_freq(m,fs,'Mensaje',t_lim=(t0,t1),f_lim=(-B,B))
        buf=io.BytesIO(); msg.export(buf,format='wav'); buf.seek(0)
        st.audio(buf.read(),format='audio/wav',sample_rate=fs)

        # 2) portadora
        t=np.arange(len(m))/fs; carrier=np.cos(2*np.pi*f0*t)
        st.subheader("2) Portadora cos(2πf0t)")
        plot_time_freq(carrier,fs,'Portadora',t_lim=(t0,t1),f_lim=(-2*f0,2*f0))

        # 3) AM
        r=(1+m/Ac)*carrier
        st.subheader("3) Señal AM r(t)")
        plot_time_freq(r,fs,'AM',t_lim=(t0,t1),f_lim=(-2*f0,2*f0))
        r_int=(r/np.max(np.abs(r))*32767).astype(np.int16)
        r_seg=AudioSegment(r_int.tobytes(),sample_width=2,frame_rate=fs,channels=1)
        buf=io.BytesIO(); r_seg.export(buf,format='wav'); buf.seek(0)
        st.audio(buf.read(),format='audio/wav',sample_rate=fs)

        # 4) mezclador y1
        y1=r*carrier
        st.subheader("4) Mezclador y1(t)")
        plot_time_freq(y1,fs,'Mezclador',t_lim=(t0,t1),f_lim=(-2*f0,2*f0))

        # 5) LPF
        Y1=np.fft.fft(y1)
        freqs=np.fft.fftfreq(len(Y1),1/fs)
        mask=np.abs(freqs)<B; y2=np.real(np.fft.ifft(Y1*mask))
        st.subheader("5) Tras LPF y2(t)")
        plot_time_freq(y2,fs,'LPF',t_lim=(t0,t1),f_lim=(-B,B))
        y2_int=(y2/np.max(np.abs(y2))*32767).astype(np.int16)
        y2_seg=AudioSegment(y2_int.tobytes(),sample_width=2,frame_rate=fs,channels=1)
        buf=io.BytesIO(); y2_seg.export(buf,format='wav'); buf.seek(0)
        st.audio(buf.read(),format='audio/wav',sample_rate=fs)

        # 6) mensaje recuperado
        m_rec=(2/Ac)*y2
        st.subheader("6) Mensaje recuperado m_rec(t)")
        plot_time_freq(m_rec,fs,'Recuperado',t_lim=(t0,t1),f_lim=(-B,B))
        m_int=(m_rec/np.max(np.abs(m_rec))*32767).astype(np.int16)
        m_seg=AudioSegment(m_int.tobytes(),sample_width=2,frame_rate=fs,channels=1)
        buf=io.BytesIO(); m_seg.export(buf,format='wav'); buf.seek(0)
        st.audio(buf.read(),format='audio/wav',sample_rate=fs)

    except Exception as ex:
        st.error(f"Error en pipeline: {ex}")
        st.text(traceback.format_exc())
else:
    st.info("👉 Ingresa URL o ID en la barra lateral.")
EOF


Detener instancias de Streamlit y revisar logs

In [46]:
!pkill -f streamlit || true
!tail -n 30 logs.txt || echo "No hay logs aún"


^C
Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


 Descargar e iniciar Cloudflared

In [47]:
# Descarga y prepara Cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# Ejecutar Streamlit en background
!streamlit run app.py &> /content/logs.txt &

# Iniciar túnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &


Extraer y mostrar la URL pública

In [48]:
import time, re, os

# Espera a que aparezca la línea en el log
time.sleep(5)

url = None
with open('/content/cloudflared.log') as f:
    for line in f:
        if "Your quick Tunnel has been created" in line:
            m = re.search(r'https?://\S+', line)
            if m:
                url = m.group(0)
                break

if url:
    print(f"✅ Tu aplicación está disponible en:\n{url}")
else:
    print("⚠️ No encontré la URL. Revisa cloudflared.log:\n")
    print(open('/content/cloudflared.log').read())

res = input("Escribe '1' para detener Streamlit: ")
if res.strip() == "1":
    os.system("pkill streamlit")
    print("🔴 Streamlit detenido.")



⚠️ No encontré la URL. Revisa cloudflared.log:

2025-06-16T06:06:49Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-06-16T06:06:49Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-06-16T06:06:52Z INF +--------------------------------------------------------------------------------------------+
2025-06-16T06:06:52Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-06-16T06:06:52Z INF |  https